# Fashion dataset
# DenseNet Application of Image Classification
## Multi label inference
The data is from the kaggle competition [iMaterialist Challenge (Fashion) at FGVC5](https://www.kaggle.com/c/imaterialist-challenge-fashion-2018)

In [1]:
from torchvision.models.densenet import densenet121 as feature_extractor
from torch import nn
import torch
import numpy as np

In [2]:
import os

## Config args

In [3]:
SCALE = 256
TRAIN = "/data/fashion/img/train/"
VALID = "/data/fashion/img/valid/"
TEST = "/data/fashion/img/test/"
CUDA = torch.cuda.is_available()
DENSE_FEATURE = 1024
BS = 32
# VERSION = "0.0.1" # bias =True for last linear
VERSION = "0.0.2" # densenet 121
# VERSION = "0.0.3" # densenet169
CATE_LEN = 228

# Mean and standard for normalization
MEAN = [0.485, 0.456, 0.406]
STD = [0.229, 0.224, 0.225]

BENCH_MARK = .5
# CUDA = False
print(CUDA)

True


In [4]:
conv_model = feature_extractor(pretrained=True)

/home/paperspace/anaconda3/envs/fastai/lib/python3.6/site-packages/torchvision/models/densenet.py:212: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  nn.init.kaiming_normal(m.weight.data)


### Split ConvLayer to 2 parts, conv0~transtition3, (not gonna train), denseblock4~norm5 (train)

In [5]:
dense_conv1 = nn.Sequential(*[getattr(conv_model.features,nn_name) for nn_name in ["conv0","norm0","relu0","pool0","denseblock1","transition1",
                                                                                   "denseblock2","transition2","denseblock3","transition3",]])

dense_conv2 = nn.Sequential(*[getattr(conv_model.features,nn_name) for nn_name in ["denseblock4","norm5"]])

### Loading data

In [6]:
from torchvision import transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader,Dataset
from torch.nn import functional as F
from PIL import Image

No augmentation here

In [7]:
transform = transforms.Compose([
                                transforms.Resize((SCALE,SCALE)),
                                transforms.ToTensor(),
                                transforms.Normalize(MEAN,STD),
                               ])

Specific data generator

In [8]:
class fashion_data(Dataset):
    def __init__(self,img_folder,cate_len,transform):
        super(fashion_data,self).__init__()
        self.img_folder = img_folder
        self.fnames = os.listdir(self.img_folder)
        self.cate_len = cate_len
        self.transform = transform
        
    def __len__(self):
        return len(self.fnames)
    
    def get_cate(self,url):
        zr = torch.zeros(228)
        zr[torch.LongTensor(list(int(i[1:])-1 for i in str(url).split(".")[0].split("_")[1:]))]=1
        return zr
    
    def __getitem__(self,idx):
        img = Image.open(self.img_folder+self.fnames[idx]).convert("RGB")
        img = self.transform(img)
        return img, self.get_cate(self.fnames[idx])

In [9]:
class test_data(Dataset):
    def __init__(self, img_folder, transform):
        self.img_folder = img_folder
        self.fnames = os.listdir(self.img_folder)
        self.transform = transform
        
    def __len__(self):
        return len(self.fnames)
    
    def __getitem__(self,idx):
        img = Image.open(self.img_folder+self.fnames[idx]).convert("RGB")
        return self.transform(img),self.fnames[idx]

In [10]:
ds = test_data("/data/fashion/img/test/",transform)

In [11]:
dl = DataLoader(ds,batch_size=BS,shuffle=False)
test_gen = iter(dl)

In [12]:
imgs,iid = next(test_gen)

#### Train /Valid dataset

### Top half of the model, with fully connected layers

In [13]:
class Flatten(nn.Module):
    def __init__(self):
        super(Flatten,self).__init__()
    def forward(self,x):
        bs = x.size()[0]
        return x.view(bs,-1)
    
def convlayer(f_in,f_out,ks,stride=1):
    return nn.Sequential(*[
        nn.Conv2d(f_in, f_out, ks, stride = stride, padding = ks//2,bias = True),
        nn.BatchNorm2d(f_out),
        nn.LeakyReLU(inplace = True),
    ])

In [14]:
fl = Flatten()
FEATURE_WIDTH = dense_conv2(dense_conv1(torch.rand(2,3,SCALE,SCALE))).size()[1]
print(FEATURE_WIDTH)

1024


In [15]:
class top_half(nn.Module):
    def __init__(self):
        super(top_half,self).__init__()
        self.top_ = nn.Sequential(*[
                                    convlayer(FEATURE_WIDTH,FEATURE_WIDTH//2,3,2),
                                    convlayer(FEATURE_WIDTH//2,FEATURE_WIDTH//4,3,2),
                                    Flatten(),
                                    nn.Linear(FEATURE_WIDTH,DENSE_FEATURE,bias=False),
                                    nn.BatchNorm1d(DENSE_FEATURE),
                                    nn.LeakyReLU(inplace=True),
                                    nn.Dropout(p=.5),
                                    nn.Linear(DENSE_FEATURE,CATE_LEN,bias=False),
                                   ])
    def forward(self,x):
        return self.top_(x)

### Construct Model, optimizer,train function

In [16]:
top_half_ = top_half()

In [17]:
if CUDA:
    torch.cuda.empty_cache()
    top_half_.cuda()
    dense_conv1.cuda()
    dense_conv2.cuda()

# loss_func = nn.CrossEntropyLoss()
# loss_func = nn.BCEWithLogitsLoss() # Binary cross entropy with logit

In [18]:
from p3self.matchbox import Trainer,argmax,accuracy,supermean,f1_score,save_model,load_model

def save_():
    save_model(dense_conv2,"/data/weights/dense_conv2.%s.pkl"%(VERSION))
    save_model(top_half_,"/data/weights/top_half.%s.pkl"%(VERSION))
    
def load_():
    load_model(dense_conv2,"/data/weights/dense_conv2.%s.pkl"%(VERSION))
    load_model(top_half_,"/data/weights/top_half.%s.pkl"%(VERSION))

def with_n(x):
    return "\n%s,"%(x)

In [19]:
IDX = torch.range(1,CATE_LEN).repeat([BS,1]).long()
LABEL_HEAD = torch.ByteTensor(np.ones((BS,1)))

In [20]:
trainer = Trainer(ds,batch_size = BS, print_on = 5, )

In [21]:
# what happened on each step of training
def action(*args,**kwargs):
    x,y = args[0]
    if CUDA:
        x,y = x.cuda(),y.cuda()
    x = x[:,:3,...]
    y_ = top_half_(dense_conv2(dense_conv1(x)))
    
    accuracy,recall,precision,f1 = f1_score(y_,y)

    return {
        "acc":accuracy.item(),
        "recall":recall.item(),
        "precision":precision.item(),
        "f1":f1.item(),
    }

In [22]:
fname = "/data/fashion/upload_%s.csv"%(VERSION)
os.system("echo 'image_id,label_id' > %s"%(fname))
def test_action(*args,**kwargs):
    x,iid = args[0]
    if CUDA:
        x = x.cuda()
    x = x[:,:3,...]
    y_ = top_half_(dense_conv2(dense_conv1(x)))
    
    iid_long = torch.LongTensor(list(int(i) for i in iid)).unsqueeze(-1)
    mark = torch.cat([iid_long,IDX[:len(iid)]],1).numpy().astype(str)
    mark[...,0]=np.vectorize(with_n)(mark[...,0])

    if CUDA:
        y_=y_.cpu()

    slic = torch.cat([LABEL_HEAD[:len(iid)],y_>BENCH_MARK],1).numpy().astype(np.bool)
    os.system("echo '%s' >> %s"%(" ".join(mark[slic].tolist()),fname))
    return {
        "ite":kwargs["ite"],
    }

In [23]:
trainer.action = test_action

### Training

Comment load_() if nothing to load

In [24]:
load_()

In [25]:
trainer.train(1)

⭐[ep_0_i_1239]	ite	1236.000: 100%|██████████| 1241/1241 [18:34<00:00,  1.11it/s]


In [26]:
torch.cuda.empty_cache()

In [27]:
f = open(fname,mode="r")
fstr = f.read()
fstr = fstr.replace("\n\n","\n").replace(", ",",")
f.close()
f2=open(fname,mode="w")


In [28]:
f2.write(fstr)

619704

In [29]:
f2.close()